# Elasticity equation - compute CFF

In this tutorial we present how to solve the elasticity equation with [PorePy](https://github.com/pmgbergen/porepy) and then how to post process the tractions to compute the CFF (Coulomb friction failure) coefficient along a fracture. The unknown is the displacement $u$.

Let $\Omega=(0,1)^3$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The CFF is computed as
$$
    CFF = \tau^\top \sigma n + \mu_f n^\top \sigma n
$$
with $\mu_f$ the fracture friction coefficient.

We will use the Multi-Point Stress Approximation (MPSA) to discretise the problem.

## Exercise 1: footing problem

The footing problem is when a force is impose on the top compressing the body and the bottom is fixed.

For this test case we set $\Omega = [0, 1]^3$, $b = 0$, and the following boundary conditions:
$$ 
u = 0 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot \sigma = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \cup \partial_{front}\Omega \cup \partial_{back} \Omega\qquad \nu \cdot \sigma = [0, -1]^\top \text{ on } \partial_{top} \Omega
$$

We have the following relations between the $\lambda$ and $\mu$ as functions of the Young's modulus $E$ and Poisson ratio $\nu$
$$
\lambda = \dfrac{\nu E}{(1+\nu)(1-2\nu)}\qquad \mu = \dfrac{E}{2(1+\nu)}
$$

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [20]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid and the fracture/fault now represent an internal constraint so that the grid adapts to it.

In [21]:
N = 10
dim = 3
mesh_size = 1 / N

# Define a fracture
frac_pts = np.array(
    [
        [0.2, 0.9, 0.9, 0.2],
        [0.2, 0.2, 0.8, 0.8],
        [0.2, 1, 1, 0.2],
    ]
)
frac = pp.PlaneFracture(frac_pts)

sd = pg.unit_grid(dim, mesh_size, as_mdg=False, fractures=[frac], constraints=[0])
sd.compute_geometry()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [22]:
key = "elasticity"

E = 1
nu = 0.25
mu_fric = 0.45

lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu = E / (2 * (1 + nu))

# Create stiffness matrix
lambda_ = lambda_ * np.ones(sd.num_cells)
mu = mu * np.ones(sd.num_cells) / 2
C = pp.FourthOrderTensor(mu, lambda_)

# Define boundary type
b_faces = sd.get_all_boundary_faces()
num_b_faces = b_faces.size
labels = np.array(["neu"] * num_b_faces)

bottom = np.isclose(sd.face_centers[2, b_faces], 0)
labels[bottom] = "dir"
bound = pp.BoundaryConditionVectorial(sd, b_faces, labels)

bc_values = np.zeros((sd.dim, sd.num_faces))

top = np.isclose(sd.face_centers[2, :], 1)
bc_values[2, top] = -sd.face_areas[top]
bc_values = bc_values.ravel("F")

# No source term
source = np.zeros(sd.num_cells * sd.dim)

# collect all data
data = {
    pp.PARAMETERS: {
        key: {
            "fourth_order_tensor": C,
            "bc_values": bc_values,
            "bc": bound,
            "source": source,
        }
    },
    pp.DISCRETIZATION_MATRICES: {key: {}},
}

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [23]:
# discretize and solve the system
mpsa = pp.Mpsa(key)
mpsa.discretize(sd, data)

A, b = mpsa.assemble_matrix_rhs(sd, data)
u = sps.linalg.spsolve(A, b)

We first compute the traction for each face of the cell and reshape it so that it has the x-components for all the faces first and then all the y-components (and then all z-components in 3d case).

In [24]:
# post process the traction for each face
mat = data[pp.DISCRETIZATION_MATRICES][key]
mat_stress = mat[mpsa.stress_matrix_key]
mat_bound_stress = mat[mpsa.bound_stress_matrix_key]

# The measure is in Pascals
t = mat_stress @ u + mat_bound_stress @ bc_values

# reshape the traction to be in the order of first all the x-components, then all the y-components
t = np.reshape(t, (sd.dim, -1), order="F").ravel()

We restrict the traction to the fracture faces

In [25]:
# compute the faces that are on the fracture
points = sd.face_centers
distances, _, _ = pp.distances.points_polygon(points, frac_pts)
faces_on_frac = np.isclose(distances, 0)

# extract the x and y components of the tractions on the fracture and make it as a vector
t_on_frac = t[np.tile(faces_on_frac, sd.dim)].reshape((sd.dim, -1))

We compute now the normal and tangential projection matrices, so that given a vector it aligns it accordingly.

In [26]:
# compute the unit normal vector to the fracture
normal = sd.face_normals[:, faces_on_frac]
normal /= np.linalg.norm(normal, axis=0)

# compute the normal and tangential projection matrix to the fracture
normal_proj = np.einsum("ik,jk->ijk", normal, normal)
tangential_proj = np.eye(3)[:, :, np.newaxis] - normal_proj

Let us compute now the normal and tangential parts of the traction and evaluate the CFF.

In [27]:
# compute the normal component of the traction
t_normal_vec = np.einsum("ijk,jk->ik", normal_proj, t_on_frac)
t_normal = np.einsum("ij,ij->j", normal, t_normal_vec)

# compute the tangential component of the traction
t_tangential_vec = np.einsum("ijk,jk->ik", tangential_proj, t_on_frac)
t_tangential = np.linalg.norm(t_tangential_vec, axis=0)

# compute the Coulomb friction factor
cff = t_tangential + mu_fric * t_normal

print(cff)

[0.00107168 0.00103937 0.00109351 0.0010725  0.00092315 0.00092937
 0.00090205 0.00089304 0.00136527 0.00093362 0.00125751 0.00141567
 0.00085157 0.00099852 0.00123214 0.00097234 0.00097466 0.00107434
 0.00084397 0.00115084 0.00109389 0.00113895 0.0011431  0.00124593
 0.00118354 0.00148279 0.00153456 0.00102791 0.00114533 0.00132727
 0.00113267 0.00092027 0.0011053  0.0011797  0.00100208 0.00088283
 0.00120198 0.00086057 0.00144951 0.00123245 0.00124161 0.00143607
 0.00125558 0.00123906 0.00143566 0.00148031 0.00099346 0.00116621
 0.00125111 0.00106379 0.00090168 0.00108289 0.00121969 0.00092794
 0.0009154  0.00104125 0.0008091  0.00110241 0.00104191 0.00107984
 0.0010912  0.00100046 0.00104954 0.0007751  0.00130405 0.00080675
 0.00085775 0.00142032 0.00140548 0.00118707 0.00115314 0.00136858
 0.00116848 0.00113782 0.00137451 0.00116252 0.00139554 0.00111658
 0.00147162 0.0012252  0.00115077 0.00119046 0.00106809 0.00095305
 0.00100929 0.00068135 0.00114674 0.00113899 0.00113625 0.0012

We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [28]:
# reshape the displacement for the export
u = np.reshape(u, (sd.dim, -1), order="F")

save = pp.Exporter(sd, "sol", folder_name="ex2")
save.write_vtu([("u", u)])

In [29]:
# Consistency check
assert np.isclose(cff.sum(), 0.1894849840616985)